**This notebook is to classify tweets if it's an emergency situation or a random tweets.**

Competition details are given here: https://www.kaggle.com/competitions/nlp-getting-started

I'll be using bidirectional LSTM model and use GloVE word embedding for creating this classification model.

Will be running this model on both CPU and GPU to compare the performance of the model for 10 iterations.

In [1]:
# Install kaggle to interact with kaggle competition and download dataset.
# Download the kaggle.json by creating a new API token from kaggle account.
!pip install kaggle

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
!cp kaggle.json ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Download and unzip the dataset
!kaggle competitions download -c nlp-getting-started
!unzip ./nlp-getting-started.zip

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 110MB/s]
Archive:  ./nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
# Download GloVE word embeddings for later use
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-08-21 21:55:18--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-08-21 21:55:19--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-08-21 21:55:19--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [4]:
# Import required libraries
import csv
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers

In [5]:
# Specify and train and test filepath
train_file = './train.csv'
test_file = './test.csv'

**Load Train and Test data from CSV**

In [6]:
# Load Training data
texts = []
labels = []

with open(train_file) as f:
  reader = csv.reader(f, delimiter=',')
  next(reader)
  for line in reader:
    texts.append(line[3])
    labels.append(line[4])

# Randomly split the train data into train and val dataset with 90:10 ratio
texts = np.array(texts)
labels = np.asarray(labels, dtype=np.int32)
totalLen = len(texts)
val_split = int(0.1 * totalLen)
val_pos = np.random.randint(totalLen, size=val_split)
train_pos = np.setdiff1d(np.arange(totalLen), val_pos)
train_sentences, train_labels = texts[train_pos], labels[train_pos]
val_sentences, val_labels = texts[val_pos], labels[val_pos]

In [7]:
# Load Test data
test_sentences = []
test_ids = []

with open(test_file) as f:
  reader = csv.reader(f, delimiter=',')
  next(reader)
  for line in reader:
    test_sentences.append(line[3])
    test_ids.append(line[0])

**Creating sequences with padding**

Convert all sentences to a fixed length, i.e. SEQ_LEN.
If sentence length is less than the SEQ_LEN, pad at the end.
Or if sentence length is greater than SEQ_LEN, truncate from the end.

In [8]:
lens = [len(sentence.split()) for sentence in train_sentences]
SEQ_LEN = int(np.percentile(lens, 100))

In [9]:
vectorizer = layers.TextVectorization(max_tokens=None,
                                      output_sequence_length=SEQ_LEN)
vectorizer.adapt(train_sentences)

**Create a dictionary of word and its corresponding indexes from vocabulary**

In [10]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

**GloVE vector of dimension 100 will be used as word embeddings**

In [11]:
path_to_glove_file = './glove.6B.100d.txt'
embedding_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, 'f', sep=' ')
    embedding_index[word] = coefs

**Create Embedding Matrix from pre-trained vectors**

Creating a map for each word in the current vocabulary, if the word is not available in GloVE then keep embedding of that matrix as sequence of zeros.

In [12]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0
missed_words = []

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
  embedding_vector = embedding_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
    hits += 1
  else:
    misses += 1
    missed_words.append(word)
print(f'Converted {hits} words and missed {misses}')

Converted 11741 words and missed 9467


In [13]:
embedding_layer = layers.Embedding(num_tokens,
                                   embedding_dim,
                                   embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                   trainable=False)

**Creating LSTM Model**


In [14]:
def create_model(lstm_units, output_units=1, output_activation='sigmoid'):
  model = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype=tf.string),
    vectorizer,
    embedding_layer,
    layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Bidirectional(layers.LSTM(lstm_units)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128),
    layers.Dropout(0.5),
    layers.Dense(units=output_units, activation=output_activation)
  ])

  return model

**Prepare dataset for optimization**

Convert an array of sentence and label to TF dataset.
Shuffle the training dataset.

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels)).shuffle(1024).batch(32).prefetch(-1)
val_dataset = tf.data.Dataset.from_tensor_slices((val_sentences, val_labels)).batch(32).prefetch(-1)

In [16]:
def train_model(model, device, epochs=10, model_callbacks=None):
  with tf.device(device_name=device):
    history = model.fit(train_dataset,
              epochs=epochs,
              validation_data=val_dataset,
              callbacks=model_callbacks)
    model.load_weights(checkpoint_filepath)
  return history

**Select a CPU and a GPU device to train the model**

In [17]:
CPU_DEVICE = '/cpu:0'
GPU_DEVICE = '/gpu:0' if tf.config.list_physical_devices('GPU') else None

**Training the Model**

First train on a CPU and then train the same model on a GPU to compare the difference in performance.

Training only for 10 iterations with same hyper-parameters.

In [18]:
# Train on CPU
checkpoint_filepath = './checkpoint_cpu'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

model_c = create_model(64, 1, 'sigmoid')
model_c.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                metrics=['accuracy'])
train_model(model_c, CPU_DEVICE, 10, [model_checkpoint_callback])

Epoch 1/10
216/216 [==============================] - 22s 73ms/step - loss: 0.9961 - accuracy: 0.5825 - val_loss: 0.5743 - val_accuracy: 0.7162
Epoch 2/10
216/216 [==============================] - 14s 66ms/step - loss: 0.8225 - accuracy: 0.6544 - val_loss: 0.5068 - val_accuracy: 0.7608
Epoch 3/10
216/216 [==============================] - 14s 65ms/step - loss: 0.7028 - accuracy: 0.7007 - val_loss: 0.4830 - val_accuracy: 0.7989
Epoch 4/10
216/216 [==============================] - 14s 66ms/step - loss: 0.6676 - accuracy: 0.7265 - val_loss: 0.4774 - val_accuracy: 0.7976
Epoch 5/10
216/216 [==============================] - 18s 82ms/step - loss: 0.6437 - accuracy: 0.7294 - val_loss: 0.4744 - val_accuracy: 0.8068
Epoch 6/10
216/216 [==============================] - 14s 66ms/step - loss: 0.6063 - accuracy: 0.7431 - val_loss: 0.4709 - val_accuracy: 0.8121
Epoch 7/10
216/216 [==============================] - 14s 65ms/step - loss: 0.5955 - accuracy: 0.7541 - val_loss: 0.4676 - val_accuracy:

In [19]:
# Train on GPU
checkpoint_filepath = './checkpoint_gpu'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

model_g = None
if GPU_DEVICE is not None:
  model_g = create_model(64, 1, 'sigmoid')
  model_g.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
  history = train_model(model_g, GPU_DEVICE, 10, [model_checkpoint_callback])

Epoch 1/10
216/216 [==============================] - 15s 22ms/step - loss: 1.0104 - accuracy: 0.5841 - val_loss: 0.5528 - val_accuracy: 0.7319
Epoch 2/10
216/216 [==============================] - 3s 16ms/step - loss: 0.8021 - accuracy: 0.6689 - val_loss: 0.5056 - val_accuracy: 0.7661
Epoch 3/10
216/216 [==============================] - 3s 16ms/step - loss: 0.7257 - accuracy: 0.7022 - val_loss: 0.5061 - val_accuracy: 0.7898
Epoch 4/10
216/216 [==============================] - 3s 15ms/step - loss: 0.6721 - accuracy: 0.7280 - val_loss: 0.5228 - val_accuracy: 0.7884
Epoch 5/10
216/216 [==============================] - 4s 17ms/step - loss: 0.6412 - accuracy: 0.7344 - val_loss: 0.4877 - val_accuracy: 0.7950
Epoch 6/10
216/216 [==============================] - 6s 26ms/step - loss: 0.6183 - accuracy: 0.7452 - val_loss: 0.4972 - val_accuracy: 0.7976
Epoch 7/10
216/216 [==============================] - 5s 21ms/step - loss: 0.5826 - accuracy: 0.7550 - val_loss: 0.4849 - val_accuracy: 0.801

**Evaluate the model on validation dataset**

In [20]:
model_g.evaluate(val_dataset)

24/24 [==============================] - 0s 7ms/step - loss: 0.4835 - accuracy: 0.8147


[0.48353710770606995, 0.8147174715995789]

In [21]:
model_c.evaluate(val_dataset)

24/24 [==============================] - 1s 8ms/step - loss: 0.4526 - accuracy: 0.8068


[0.45262742042541504, 0.8068330883979797]

**Optional steps to submit the predicted results to the kaggle competition**

In [22]:
preds = model_g.predict(test_sentences)

In [23]:
test_pred_labels = tf.cast(tf.squeeze(tf.round(preds)), dtype=tf.int32)

In [24]:
import pandas as pd

test_df = pd.DataFrame({
    'id': test_ids,
    'target': test_pred_labels
})
test_df.to_csv('./submission_gpu.csv', index=False)

In [25]:
!kaggle competitions submit -c nlp-getting-started -f ./submission_gpu.csv -m "gpu"

100% 22.2k/22.2k [00:02<00:00, 9.11kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets